## Código

In [ ]:
import tweepy
import pandas as pd
import os
import IPython.display as display  # Para mostrar tablas

# Leer token.txt
token_path = "bearer_token.txt"
try:
    with open(token_path, "r") as f:
        BEARER_TOKEN = f.read().strip()
    print("Token cargado correctamente.")
except FileNotFoundError:
    raise FileNotFoundError(f"\u26a0\ufe0f No se encontró el archivo '{token_path}'. Asegúrate de haberlo guardado antes.")

# Crear el cliente de Tweepy
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Definir la consulta
query = "(hurricane OR earthquake OR flood OR tornado OR tsunami OR wildfire OR landslide OR avalanche OR typhoon OR cyclone OR eruption OR blizzard) -is:retweet lang:en"

# Archivo que almacenará los IDs de tweets ya procesados
processed_ids_file = "processed_ids.txt"
if os.path.exists(processed_ids_file):
    with open(processed_ids_file, "r") as f:
        processed_ids = {line.strip() for line in f if line.strip()}
    print(f"IDs ya procesados: {processed_ids}")
else:
    processed_ids = set()
    print("No se encontró archivo de IDs procesados, se inicia con un conjunto vacío.")

# Realizar la búsqueda de tweets recientes sin usar 'since_id'
response = client.search_recent_tweets(
    query=query,
    tweet_fields=["author_id", "created_at", "id", "geo"],
    user_fields=["location", "verified"],
    expansions=["author_id"],
    max_results=100
)

if response.data:
    # Filtrar solo los tweets cuyos IDs no estén ya procesados
    new_tweets = [tweet for tweet in response.data if str(tweet.id) not in processed_ids]

    if new_tweets:
        # Obtener datos de los usuarios si están disponibles
        users = {user.id: user for user in response.includes["users"]} if response.includes else {}

        # Preparar la información de cada tweet
        tweet_records = []
        for tweet in new_tweets:
            record = {
                "TweetID": tweet.id,
                "AuthorID": tweet.author_id,
                "CreatedAt": tweet.created_at.isoformat() if tweet.created_at else "",
                "Location": (users.get(tweet.author_id, {}).location 
                             if tweet.author_id in users and hasattr(users[tweet.author_id], "location") 
                             else "Desconocida"),
                "Country": (users.get(tweet.author_id, {}).geo 
                            if tweet.author_id in users and hasattr(users[tweet.author_id], "geo") 
                            else "Desconocido"),
                "Description": (users[tweet.author_id].description 
                                if tweet.author_id in users and hasattr(users[tweet.author_id], "description")
                                else "No disponible"),
                "Verified": (users.get(tweet.author_id, {}).verified 
                             if tweet.author_id in users and hasattr(users[tweet.author_id], "verified") 
                             else "Desconocido"),
                "Text": tweet.text
            }
            tweet_records.append(record)

        df_new = pd.DataFrame(tweet_records)

        tweets_file = "tweets.csv"

        # Concatenar nuevos tweets si ya existe el archivo
        if os.path.exists(tweets_file):
            df_existing = pd.read_csv(tweets_file)
            df_total = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_total = df_new

        # Guardar (o actualizar) el archivo CSV
        df_total.to_csv(tweets_file, index=False)
        print(f"\n\u2705 Tweets guardados en '{tweets_file}'")

        # Actualizar el archivo de IDs con los nuevos tweets procesados
        with open(processed_ids_file, "a") as f:
            for tweet in new_tweets:
                tweet_id_str = str(tweet.id)
                if tweet_id_str not in processed_ids:
                    f.write(tweet_id_str + "\n")
                    processed_ids.add(tweet_id_str)
        print("IDs procesados actualizados.")
    else:
        print("No hay nuevos tweets para agregar (todos ya fueron registrados).")
else:
    print("No se encontraron tweets con la consulta indicada.")


Token cargado correctamente.
IDs ya procesados: {'1890480025100529825', '1890087351789207867', '1889775381479067830', '1890013345568436246', '1891589156624601263', '1889766982116868244', '1889996464216244560', '1889476378740793393', '1889797904979915119', '1891405147302990126', '1889946766810009752', '1890414131649372246', '1891588644013572331'}

✅ Tweets guardados en 'tweets.csv'
IDs procesados actualizados.
